In [1]:
!pip install azure-storage-blob # Microoft Azure
!pip install pyarrow
!pip install psycopg2 sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.4 MB/s eta 0:00:00


In [2]:
# import libraries
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from math import ceil
import datetime
import calendar
from sqlalchemy import create_engine

In [3]:
# Function
def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    blob_content = download_stream.readall().decode('utf-8')
    return blob_content

### Get raw_NYCRest.csv from Azure

In [5]:
config_file_path = 'config.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

connection_string = config["connectionString"]
container_name = "groupproject"
blob_name = "groupdata2_raw_NYCRest.csv"

blob_content = azure_download_blob(connection_string, container_name, blob_name)

raw_NYCRest_df = pd.read_csv(StringIO(blob_content))
raw_NYCRest_df

,camis,dba,boro,building,street,zipcode,phone,inspection_date,critical_flag,record_date,latitude,longitude,cuisine_description,action,score,inspection_type,violation_code,violation_description,grade,grade_date
0,41022989,MULBERRY STREET BAR,Manhattan,176 1/2,MULBERRY STREET,10013.0,2122269345,2021-09-17T00:00:00.000,Not Applicable,2024-04-16T06:00:10.000,40.720320,-73.996894,Italian,Violations were cited in the following area(s).,0.0,Inter-Agency Task Force / Initial Inspection,NaN,NaN,NaN,NaN
1,50061025,SUBWAY,Manhattan,21,MAIDEN LANE,10038.0,2125714417,2022-06-22T00:00:00.000,Not Critical,2024-04-16T06:00:10.000,40.709253,-74.009082,Sandwiches,Violations were cited in the following area(s).,46.0,Cycle Inspection / Initial Inspection,10C,Lighting Inadequate,NaN,NaN
2,50103012,TIFFANY CAFE,Manhattan,6,EAST 57 STREET,10022.0,2126135511,2023-03-29T00:00:00.000,Not Applicable,2024-04-16T06:00:10.000,40.762773,-73.973547,American,No violations were recorded at the time of thi...,0.0,Cycle Inspection / Initial Inspection,NaN,NaN,A,2023-03-29T00:00:00.000
3,50104962,EMPANADAS RD,Bronx,1971,DR MARTIN L KING JR BLVE,NaN,3477264990,2021-10-08T00:00:00.000,Not Critical,2024-04-16T06:00:10.000,0.000000,0.000000,Latin American,Violations were cited in the following area(s).,NaN,Administrative Miscellaneous / Re-inspection,22F,MISBRANDED AND LABELING,NaN,NaN
4,50016699,TU CASA,Queens,10311,QUEENS BLVD,11375.0,7182752421,2022-04-26T00:00:00.000,Critical,2024-04-16T06:00:10.000,40.724745,-73.850128,Spanish,Violations were cited in the following area(s).,11.0,Cycle Inspection / Initial Inspection,02B,Hot food item not held at or above 140º F.,A,2022-04-26T00:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221974,50084577,FARM TO BURGER,Manhattan,310,WEST 40 STREET,10018.0,6466095122,2023-02-10T00:00:00.000,Not Critical,2024-04-16T06:00:10.000,40.756202,-73.991359,American,Violations were cited in the following area(s).,11.0,Cycle Inspection / Re-inspection,10F,Non-food contact surface or equipment made of ...,A,2023-02-10T00:00:00.000
221975,50051743,ORCHARD BEACH SNACK BAR SECTION 10,Bronx,1,ORCHARD BEACH ROAD,10464.0,6462294471,2017-06-29T00:00:00.000,Critical,2024-04-16T06:00:10.000,40.865153,-73.802838,Coffee/Tea,Violations were cited in the following area(s).,11.0,Cycle Inspection / Initial Inspection,04K,Evidence of rats or live rats present in facil...,A,2017-06-29T00:00:00.000
221976,50088485,VAN LEEUWEN ICE CREAM,Brooklyn,81,BERGEN STREET,11201.0,7187011630,2023-03-27T00:00:00.000,Not Critical,2024-04-16T06:00:10.000,40.686556,-73.990023,Frozen Desserts,Violations were cited in the following area(s).,8.0,Cycle Inspection / Re-inspection,10G,Dishwashing and ware washing: Cleaning and sa...,A,2023-03-27T00:00:00.000
221977,41309636,LONA PIZZA,Bronx,373,EAST 204 STREET,10467.0,7186556600,2022-03-17T00:00:00.000,Critical,2024-04-16T06:00:10.000,40.871567,-73.877236,Pizza,Violations were cited in the following area(s).,33.0,Cycle Inspection / Initial Inspection,02B,Hot food item not held at or above 140º F.,NaN,NaN


In [6]:
raw_NYCRest_df.columns

Index(['camis', 'dba', 'boro', 'building', 'street', 'zipcode', 'phone',
       'inspection_date', 'critical_flag', 'record_date', 'latitude',
       'longitude', 'cuisine_description', 'action', 'score',
       'inspection_type', 'violation_code', 'violation_description', 'grade',
       'grade_date'],
      dtype='object')

In [7]:
# Specify the columns you want to keep
columns_to_keep = [
    'dba', 'boro', 'building', 'street', 'zipcode', 'phone', 'inspection_date', 'critical_flag', 'cuisine_description', 'action',
    'score', 'inspection_type', 'violation_code', 'violation_description', 'grade', 'grade_date', 'latitude', 'longitude'
]

# Filter the DataFrame to include only the columns of interest
raw_NYCRest_df_filtered = raw_NYCRest_df[columns_to_keep]

raw_NYCRest_df_filtered

,dba,boro,building,street,zipcode,phone,inspection_date,critical_flag,cuisine_description,action,score,inspection_type,violation_code,violation_description,grade,grade_date,latitude,longitude
0,MULBERRY STREET BAR,Manhattan,176 1/2,MULBERRY STREET,10013.0,2122269345,2021-09-17T00:00:00.000,Not Applicable,Italian,Violations were cited in the following area(s).,0.0,Inter-Agency Task Force / Initial Inspection,NaN,NaN,NaN,NaN,40.720320,-73.996894
1,SUBWAY,Manhattan,21,MAIDEN LANE,10038.0,2125714417,2022-06-22T00:00:00.000,Not Critical,Sandwiches,Violations were cited in the following area(s).,46.0,Cycle Inspection / Initial Inspection,10C,Lighting Inadequate,NaN,NaN,40.709253,-74.009082
2,TIFFANY CAFE,Manhattan,6,EAST 57 STREET,10022.0,2126135511,2023-03-29T00:00:00.000,Not Applicable,American,No violations were recorded at the time of thi...,0.0,Cycle Inspection / Initial Inspection,NaN,NaN,A,2023-03-29T00:00:00.000,40.762773,-73.973547
3,EMPANADAS RD,Bronx,1971,DR MARTIN L KING JR BLVE,NaN,3477264990,2021-10-08T00:00:00.000,Not Critical,Latin American,Violations were cited in the following area(s).,NaN,Administrative Miscellaneous / Re-inspection,22F,MISBRANDED AND LABELING,NaN,NaN,0.000000,0.000000
4,TU CASA,Queens,10311,QUEENS BLVD,11375.0,7182752421,2022-04-26T00:00:00.000,Critical,Spanish,Violations were cited in the following area(s).,11.0,Cycle Inspection / Initial Inspection,02B,Hot food item not held at or above 140º F.,A,2022-04-26T00:00:00.000,40.724745,-73.850128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221974,FARM TO BURGER,Manhattan,310,WEST 40 STREET,10018.0,6466095122,2023-02-10T00:00:00.000,Not Critical,American,Violations were cited in the following area(s).,11.0,Cycle Inspection / Re-inspection,10F,Non-food contact surface or equipment made of ...,A,2023-02-10T00:00:00.000,40.756202,-73.991359
221975,ORCHARD BEACH SNACK BAR SECTION 10,Bronx,1,ORCHARD BEACH ROAD,10464.0,6462294471,2017-06-29T00:00:00.000,Critical,Coffee/Tea,Violations were cited in the following area(s).,11.0,Cycle Inspection / Initial Inspection,04K,Evidence of rats or live rats present in facil...,A,2017-06-29T00:00:00.000,40.865153,-73.802838
221976,VAN LEEUWEN ICE CREAM,Brooklyn,81,BERGEN STREET,11201.0,7187011630,2023-03-27T00:00:00.000,Not Critical,Frozen Desserts,Violations were cited in the following area(s).,8.0,Cycle Inspection / Re-inspection,10G,Dishwashing and ware washing: Cleaning and sa...,A,2023-03-27T00:00:00.000,40.686556,-73.990023
221977,LONA PIZZA,Bronx,373,EAST 204 STREET,10467.0,7186556600,2022-03-17T00:00:00.000,Critical,Pizza,Violations were cited in the following area(s).,33.0,Cycle Inspection / Initial Inspection,02B,Hot food item not held at or above 140º F.,NaN,NaN,40.871567,-73.877236


In [8]:
# split the dataframe based on Boro
df_brooklyn = raw_NYCRest_df_filtered[raw_NYCRest_df_filtered['boro'] == 'Brooklyn']
df_queens = raw_NYCRest_df_filtered[raw_NYCRest_df_filtered['boro'] == 'Queens']
df_manhattan = raw_NYCRest_df_filtered[raw_NYCRest_df_filtered['boro'] == 'Manhattan']
df_bronx = raw_NYCRest_df_filtered[raw_NYCRest_df_filtered['boro'] == 'Bronx']
df_statenisland = raw_NYCRest_df_filtered[raw_NYCRest_df_filtered['boro'] == 'Staten Island']

In [ ]:
df_bronx

In [ ]:
df_bronx.columns

### Get Yelp.csv from Azure

In [10]:
config_file_path = 'config.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

connection_string = config["connectionString"]
container_name = "groupproject"
blob_name = "groupdata3_Yelp.csv"

blob_content = azure_download_blob(connection_string, container_name, blob_name)


yelp_df = pd.read_csv(StringIO(blob_content))
yelp_df

,dba,boro,building,street,zipcode,phone,cuisine_description,latitude,longitude,yelp_rating,yelp_review_count
0,NONIS COFFEE SHOP,Bronx,5921,RIVERDALE AVENUE,10471.0,7186010032,American,40.907329,-73.903981,3.5,32.0
1,RIVERDALE STEAK HOUSE,Bronx,5700,RIVERDALE AVENUE,10471.0,7185499946,Steakhouse,40.906719,-73.904123,3.6,72.0
2,SKYCLUB CAFE,Bronx,5701,ARLINGTON AVENUE,10471.0,2016169579,American,40.906648,-73.906844,5.0,1.0
3,CARLOS AND GABBY'S,Bronx,5685,RIVERDALE AVENUE,10471.0,6463610119,Jewish/Kosher,40.906481,-73.904210,3.2,81.0
4,THE PIZZA BLOCK,Bronx,5677,RIVERDALE AVENUE,10471.0,7187087004,Jewish/Kosher,40.906341,-73.904247,2.1,16.0
...,...,...,...,...,...,...,...,...,...,...,...
1346,LOS HERMANOS SALAZAR RESTAURANT,Bronx,645,COURTLANDT AVENUE,10451.0,9174718320,Mexican,40.818888,-73.918492,2.6,71.0
1347,GLACKEN'S BAR,Bronx,135,EAST 149 STREET,10451.0,7185857082,American,40.818870,-73.928497,3.9,15.0
1348,AMERICAN GRILL,Bronx,2987,3 AVENUE,10455.0,7185856978,American,40.818847,-73.914142,3.0,2.0
1349,BRIDGES BAR,Bronx,4100,EAST TREMONT AVENUE,10465.0,3473989144,American,40.818818,-73.817114,3.0,1.0


### Create the Final DataFrame with all needed attributes

In [ ]:
merged_df = df_bronx.merge(yelp_df, left_on=['dba'], right_on=['dba'], how='left')
merged_df

In [ ]:
df_bronx_final = merged_df.copy()
df_bronx_final.columns

In [ ]:
df_bronx_final = df_bronx_final[['dba', 'boro_x', 'building_x', 'street_x', 'zipcode_x', 'phone_x',
       'inspection_date', 'critical_flag', 'cuisine_description_x', 'action',
       'score', 'inspection_type', 'violation_code', 'violation_description',
       'grade', 'grade_date', 'latitude_x', 'longitude_x', 'yelp_rating',
       'yelp_review_count']]

In [ ]:
df_bronx_final

In [ ]:
df_bronx_final.rename(columns={'boro_x': 'boro', 'building_x': 'building', 'street_x': 'street', 'zipcode_x': 'zipcode', 'phone_x': 'phone', 'cuisine_description_x': 'cuisine_description', 'latitude_x': 'latitude', 'longitude_x': 'longitude'}, inplace=True)
df_bronx_final

In [ ]:
df_bronx_final.dropna(subset=['yelp_rating'], inplace=True)
df_bronx_final['inspection_date'] = pd.to_datetime(df_bronx_final['inspection_date'])
df_bronx_final['grade_date'] = pd.to_datetime(df_bronx_final['grade_date'])

In [ ]:
df_bronx_final

,dba,boro,building,street,zipcode,phone,inspection_date,critical_flag,cuisine_description,action,score,inspection_type,violation_code,violation_description,grade,grade_date,latitude,longitude,yelp_rating,yelp_review_count
3,CORKY'S DINER,Bronx,2535,GRAND CONCOURSE,10468.0,7189332484,2024-01-24,Critical,American,Violations were cited in the following area(s).,18.0,Cycle Inspection / Initial Inspection,02B,Hot TCS food item not held at or above 140 °F.,NaN,NaT,40.863278,-73.896514,3.0,113.0
4,PAPA JOHN'S (STAND 310),Bronx,1,EAST 161 STREET,10451.0,9172843260,2017-07-25,Not Applicable,Pizza,No violations were recorded at the time of thi...,0.0,Cycle Inspection / Initial Inspection,NaN,NaN,A,2017-07-25,40.829028,-73.928496,1.9,24.0
5,JADE PALACE,Bronx,163,EINSTEIN LOOP,10475.0,7183201584,2022-03-23,Critical,Chinese,Violations were cited in the following area(s).,31.0,Cycle Inspection / Initial Inspection,02B,Hot food item not held at or above 140º F.,NaN,NaT,40.864063,-73.822546,2.6,11.0
6,PINE BAR & GRILL,Bronx,1634,EASTCHESTER ROAD,10461.0,7183190900,2017-10-12,Critical,Italian,Violations were cited in the following area(s).,12.0,Cycle Inspection / Re-inspection,06F,Wiping cloths soiled or not stored in sanitizi...,A,2017-10-12,40.845277,-73.845095,3.0,2.0
7,LA ROLA RESTAURANT,Bronx,400,EAST 198 STREET,10458.0,9176881449,2024-03-06,Not Applicable,Spanish,Establishment re-opened by DOHMH.,0.0,Cycle Inspection / Reopening Inspection,NaN,NaN,Z,2024-03-06,40.866021,-73.886021,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19571,XIN HI CHINESE BUFFETT,Bronx,2053,BARTOW AVENUE,10475.0,7183792200,2023-01-20,Not Critical,Chinese,Violations were cited in the following area(s).,5.0,Cycle Inspection / Re-inspection,28-06,Contract with a pest management professional n...,A,2023-01-20,40.869468,-73.828515,4.0,5.0
19572,PROSPECT COFFEE SHOP,Bronx,1309,PROSPECT AVENUE,10459.0,7189911249,2023-03-20,Not Critical,American,Violations were cited in the following area(s).,9.0,Cycle Inspection / Initial Inspection,10F,Non-food contact surface or equipment made of ...,A,2023-03-20,40.828042,-73.898148,0.0,0.0
19577,"SAKE II JAPANESE RESTAURANT, SUSHI & HIBACHI",Bronx,690,EAST 187 STREET,10458.0,7182200988,2023-01-11,Critical,Japanese,Violations were cited in the following area(s).,30.0,Cycle Inspection / Re-inspection,04H,"Raw, cooked or prepared food is adulterated, c...",C,2023-01-11,40.854192,-73.884643,2.7,3.0
19579,FOO-HING KITCHEN,Bronx,2895,SEDGWICK AVENUE,10468.0,7188846267,2021-09-07,Critical,Chinese,Violations were cited in the following area(s).,22.0,Cycle Inspection / Initial Inspection,06B,"Tobacco use, eating, or drinking from open con...",NaN,NaT,40.874944,-73.901262,0.0,0.0


### Upload Final DataFrame to Azure (groupdata4_Merge_df_Bronx.csv)

In [ ]:
config_file_path = 'config.json'

with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
CONTAINER_AZURE = "groupproject"
blob_name = "groupdata4_Merge_df_Bronx.csv"

# Convert DataFrame to CSV
output = StringIO()
df_bronx_final.to_csv(output, index=False)
data = output.getvalue()
output.close()

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get a blob client using the container name and blob name
blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

# Upload the CSV data
blob_client.upload_blob(data, overwrite=True)

print(f"Uploaded {blob_name} to Azure Blob Storage in container {CONTAINER_AZURE}.")

Uploaded groupdata4_Merge_df_Bronx.csv to Azure Blob Storage in container groupproject.
